In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
from cmocean import cm
from cartopy.crs import LambertAzimuthalEqualArea, PlateCarree
from cartopy.feature import LAND

%matplotlib inline

In [ ]:
date0 = '19910915'
date1 = '19920101'
date2 = '19920915'


myi_file0 = f'../data2/Anton/sia/cdr_1991_2023/sic/1991/sic_{date0}_{date0}.npz'
sic_file1 = f'../data2/Anton/sia/cdr_1991_2023/sic/1992/sic_{date1}.npz'
myi_file2 = f'../data2/Anton/sia/cdr_1991_2023/sic/1992/sic_{date2}_{date2}.npz'

ca0_file1 = f'../data2/Anton/sia/cdr_1991_2023/sic/1991/sic_{date0}_{date1}.npz'
ca1_file2 = f'../data2/Anton/sia/cdr_1991_2023/sic/1991/sic_{date0}_{date2}.npz'

msh_file0 = f'../data2/Anton/sia/cdr_1991_2023/mesh/1991/mesh_{date0}.npz'
msh_file1 = f'../data2/Anton/sia/cdr_1991_2023/mesh/1992/mesh_{date1}.npz'
msh_file2 = f'../data2/Anton/sia/cdr_1991_2023/mesh/1992/mesh_{date2}.npz'

sic_files = [myi_file0, ca0_file1, sic_file1, ca1_file2, myi_file2]
msh_files = [msh_file0, msh_file1, msh_file1, msh_file2, msh_file2]
titles = ['$C_{MYI}$ 1991-09-15', '$C_{Ao}$ 1992-01-01', '$C_{TOT}$ 1992-01-01', '$C_{A1}$ 1992-09-15', '$C_{MYI}$ 1992-09-15']

srs_dst = LambertAzimuthalEqualArea(central_longitude=0, central_latitude=90)

In [ ]:
def plot_sic(x, y, t, c, title, ofile, cmap=cm.ice):
    fig = plt.figure(figsize=(4, 5))
    ax = fig.add_subplot(1, 1, 1, projection=srs_dst)
    tcf = ax.tripcolor(x, y, t, c, cmap=cmap, zorder=0, clim=[0, 100])
    ax.add_feature(LAND, facecolor='lightgray', edgecolor='black', zorder=1)
    ax.set_extent([-1700000, 2000000, -2000000, 2200000], crs=srs_dst)
    plt.title(title, fontsize=26)
    plt.tight_layout()
    plt.savefig(ofile, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()
    print(ofile)

for sic_file, msh_file, title in zip(sic_files, msh_files, titles):
    c = np.load(sic_file)['c']
    with np.load(msh_file) as ds:
        x = ds['x']*1000
        y = ds['y']*1000
        t = ds['t']
    ofile = os.path.basename(sic_file).replace('.npz', '.png')
    plot_sic(x, y, t, c, title, ofile)


In [ ]:
c_tot = np.load(sic_file1)['c']
c_ca0 = np.load(ca0_file1)['c']
c_1iy = c_tot - c_ca0
with np.load(msh_file1) as ds:
    x = ds['x']*1000
    y = ds['y']*1000
    t = ds['t']
plot_sic(x, y, t, c_1iy, '$C_{1Y}$ 1992-01-01', f'sic_1yi_{date1}.png', 'viridis')

In [ ]:
c_myi = np.load(myi_file2)['c']
c_ca1 = np.load(ca1_file2)['c']

c_2iy = c_myi - c_ca1
with np.load(msh_file2) as ds:
    x = ds['x']*1000
    y = ds['y']*1000
    t = ds['t']

plot_sic(x, y, t, c_2iy, '$C_{2Y}$ 1992-09-15', f'sic_2yi_{date2}.png', 'viridis')


In [ ]:
lon, lat = PlateCarree().transform_points(srs_dst, x, y).T[:2]
plt.plot(lon, lat, 'k.', markersize=1)